# 基本信息
1. 实验名称：网络优化实验
2. 姓名：戴斌斌
3. 学号：20281239
4. 日期：2022/11/14

---

# 一、任务1-在多分类任务实验中手动实现dropout

## 1.1 任务内容

1. 任务具体要求  
在多分类任务实验中分别手动实现dropout  
探究不同丢弃率对实验结果的影响（可用loss曲线进行展示）
2. 任务目的  
探究不同丢弃率对实验结果的影响
3. 任务算法或原理介绍    
Dropout 原理   
![](https://drailife.oss-cn-beijing.aliyuncs.com/img/202211112255396.png)
4. 任务所用数据集   
   MNIST手写体数据集:  
     + 该数据集包含60,000个用于训练的图像样本和10,000个用于测试的图像样本。  
     + 图像是固定大小(28x28像素)，其值为0到1。为每个图像都被平展并转换为784  
        
## 1.2 任务思路及代码  

1. 构建数据集
2. 构建前馈神经网络，损失函数，优化函数
3. 手动实现dropout
3. 使用网络预测结果，得到损失值  
4. 进行反向传播，和梯度更新  
5. 对loss、acc等指标进行分析

### 1.2.0数据集定义

In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torch.nn.functional import cross_entropy, binary_cross_entropy
from torch.nn import CrossEntropyLoss
from torchvision import transforms
from sklearn import  metrics
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 如果有gpu则在gpu上计算 加快计算速度
print(f'当前使用的device为{device}')
# 数据集定义
# 定义多分类数据集 - train_dataloader - test_dataloader
batch_size = 128
# Build the training and testing dataset
traindataset = torchvision.datasets.FashionMNIST(root='E:\\DataSet\\FashionMNIST\\Train',
                                                  train=True,
                                                  download=True,
                                                  transform=transforms.ToTensor())
testdataset = torchvision.datasets.FashionMNIST(root='E:\\DataSet\\FashionMNIST\\Test',
                                                 train=False,
                                                 download=True,
                                                 transform=transforms.ToTensor())
traindataloader = torch.utils.data.DataLoader(traindataset, batch_size=batch_size, shuffle=True)
testdataloader = torch.utils.data.DataLoader(testdataset, batch_size=batch_size, shuffle=False)
# 绘制图像的代码
def picture(name, trainl, testl, type='Loss'):
    plt.rcParams["font.sans-serif"]=["SimHei"] #设置字体
    plt.rcParams["axes.unicode_minus"]=False #该语句解决图像中的“-”负号的乱码问题
    plt.title(name) # 命名
    plt.plot(trainl, c='g', label='Train '+ type)
    plt.plot(testl, c='r', label='Test '+type)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
print(f'多分类数据集 样本总数量{len(traindataset) + len(testdataset)},训练样本数量{len(traindataset)},测试样本数量{len(testdataset)}')

当前使用的device为cuda
多分类数据集 样本总数量70000,训练样本数量60000,测试样本数量10000


In [2]:
# 定义自己的前馈神经网络
class MyNet():
    def __init__(self,dropout=0):
        # 设置隐藏层和输出层的节点数
        self.dropout = dropout
        self.is_train = None
        num_inputs, num_hiddens, num_outputs = 28 * 28, 256, 10  # 十分类问题
        w_1 = torch.tensor(np.random.normal(0, 0.01, (num_hiddens, num_inputs)), dtype=torch.float32,
                           requires_grad=True)
        b_1 = torch.zeros(num_hiddens, dtype=torch.float32, requires_grad=True)
        w_2 = torch.tensor(np.random.normal(0, 0.01, (num_outputs, num_hiddens)), dtype=torch.float32,
                           requires_grad=True)
        b_2 = torch.zeros(num_outputs, dtype=torch.float32, requires_grad=True)
        self.params = [w_1, b_1, w_2, b_2]

        # 定义模型结构
        self.input_layer = lambda x: x.view(x.shape[0], -1)
        self.hidden_layer = lambda x: self.my_relu(torch.matmul(x, w_1.t()) + b_1)
        self.output_layer = lambda x: torch.matmul(x, w_2.t()) + b_2
    
    def my_relu(self, x):
        return torch.max(input=x, other=torch.tensor(0.0))
    # 以下两个函数分别在训练和测试前调用，选择是否需要dropout
    def train(self):
        self.is_train = True
    def test(self):
        self.is_train = False
    # 定义前向传播
    def forward(self, x):
        x = self.input_layer(x)
        if self.is_train:
            x = dropout_layer(x,dropout=self.dropout)
        x = self.hidden_layer(x)
        if self.is_train:
            x = dropout_layer(x,dropout=self.dropout)
        x = self.output_layer(x)
        return x
"""
定义dropout层
x: 输入数据
dropout: 随机丢弃的概率
"""
def dropout_layer(x, dropout):
    assert 0 <= dropout <= 1 #dropout值必须在0-1之间
    # dropout==1，所有元素都被丢弃。
    if dropout == 1:
        return torch.zeros_like(x)
        # 在本情况中，所有元素都被保留。
    if dropout == 0:
        return x
    mask = (torch.rand(x.shape) > dropout).float() #rand()返回一个张量，包含了从区间[0, 1)的均匀分布中抽取的一组随机数
    return mask * x / (1.0 - dropout)

# 默认的优化函数为手写的mySGD
def mySGD(params, lr, batchsize):
    for param in params:
        param.data -= lr * param.grad / batchsize
"""
定义训练函数
model:定义的模型 默认为MyNet(0) 即无dropout的初始网络
epochs:训练总轮数 默认为40
criterion:定义的损失函数，默认为cross_entropy
lr :学习率 默认为0.01
optimizer:定义的优化函数，默认为自己定义的mySGD函数
"""
def train_and_test(model=MyNet(),epochs=40,criterion = cross_entropy,lr=0.01,optimizer=mySGD):
    train_all_loss = []  # 记录训练集上得loss变化
    test_all_loss = []  # 记录测试集上的loss变化
    train_ACC, test_ACC = [], [] # 记录正确的个数
    begintime = time.time()
    model.train() #表明当前处于训练状态，允许使用dropout
    for epoch in range(epochs):
        train_l,train_acc_num = 0, 0
        for data, labels in traindataloader:
            pred = model.forward(data)
            train_each_loss = criterion(pred, labels)  # 计算每次的损失值
            train_l += train_each_loss.item()
            train_each_loss.backward()  # 反向传播
            optimizer(model.params, lr, 128)  # 使用小批量随机梯度下降迭代模型参数
            # 梯度清零
            train_acc_num += (pred.argmax(dim=1)==labels).sum().item()
            for param in model.params:
                param.grad.data.zero_()
            # print(train_each_loss)
        train_all_loss.append(train_l)  # 添加损失值到列表中
        train_ACC.append(train_acc_num / len(traindataset)) # 添加准确率到列表中
        model.test() # 表明当前处于测试状态，无需使用dropout
        with torch.no_grad():
            is_train = False  # 表明当前为测试阶段，不需要dropout参与
            test_l, test_acc_num = 0, 0
            for data, labels in testdataloader:
                pred = model.forward(data)
                test_each_loss = criterion(pred, labels)
                test_l += test_each_loss.item()
                test_acc_num += (pred.argmax(dim=1)==labels).sum().item()
            test_all_loss.append(test_l)
            test_ACC.append(test_acc_num / len(testdataset))   # # 添加准确率到列表中
        if epoch == 0 or (epoch + 1) % 4 == 0:
            print('epoch: %d | train loss:%.5f | test loss:%.5f | train acc: %.2f | test acc: %.2f'
                  % (epoch + 1, train_l, test_l, train_ACC[-1],test_ACC[-1]))
    endtime = time.time()
    print("手动实现dropout = 0.2 %d轮 总用时: %.3f" % (epochs, endtime - begintime))
    return train_all_loss,test_all_loss,train_ACC,test_ACC

### 1.2.1 手动实验-设置dropout = 0

In [ ]:
"""
    设置dropout = 0  dropout = 0  epoch = 40  lr = 0.10  optimizer = mySGD
"""
model_11 = MyNet(dropout=0)
train_and_test(model=model_11,epochs=40,lr=0.01,optimizer=mySGD)

### 1.2.2 手动实验-设置dropout = 0.3  

In [ ]:
"""
    设置dropout = 0.3  epoch = 40  lr = 0.01  optimizer = mySGD
"""
model_12 = MyNet(dropout=0.3)
train_and_test(model=model_12,epochs=40,lr=0.01,optimizer=mySGD)

### 1.2.3 手动实验-设置dropout = 0.6

In [ ]:
"""
    设置dropout = 0.6  dropout = 0.6  epoch = 40  lr = 0.01  optimizer = mySGD
"""
model_13 = MyNet(dropout=0.6)
train_and_test(model=model_13,epochs=40,lr=0.01,optimizer=mySGD)

### 1.2.4 手动实验-设置dropout = 0.9

In [ ]:
"""
    设置dropout = 0.9  dropout = 0.9  epoch = 40  lr = 0.05  optimizer = mySGD
"""
model_14 = MyNet(dropout=0.9)
train_and_test(model=model_14,epochs=40,lr=0.05,optimizer=mySGD)

## 1.3 实验结果分析  
将上述前馈网络回归任务每一轮得训练和测试得损失值绘制成图表，如下图：

将上述的二分类和多分类的正确率绘制成表格

---

### 2.2.1 torch.nn实现前馈神经网络-回归任务

### 2.2.2 torch.nn实现前馈神经网络-二分类

### 2.2.3 torch.nn实现前馈神经网络-多分类任务

## 2.3 实验结果分析  
将上述前馈网络回归任务每一轮得训练和测试得损失值绘制成图表，如下图：

In [ ]:
plt.figure(figsize=(12,3))
plt.subplot(131)
picture('前馈网络-回归-loss',train_all_loss21,test_all_loss21)
plt.subplot(132)
picture('前馈网络-二分类-loss',train_all_loss22,test_all_loss22)
plt.subplot(133)
picture('前馈网络-多分类-loss',train_all_loss23,test_all_loss23)
plt.show()

将上述前馈网络回归任务每一轮得训练和测试得准确率值绘制成图表，如下图：

In [ ]:
plt.figure(figsize=(8,3))
plt.subplot(121)
picture('前馈网络-二分类-ACC',train_ACC22,test_ACC22,type='ACC')
plt.subplot(122)
picture('前馈网络-多分类-ACC',train_ACC23,test_ACC23,type='ACC')
plt.show()

torch.nn实现的前馈网络在不同的数据集上表现出不同的效果，在回归问题中，损失函数下降速度极快，在前几轮就充分训练模型，并取得较好的效果。  
在二分类中，loss值稳步下降，准确率逐步提升。  
在多分类中，数据的规模较大，loss值在前6轮中下降较快，准确率提升较大，在后续的训练中，loss值相比前几轮下降较慢，准确率提升速度也不及前几轮。同时由于训练轮数的不足，模型并未没被完全训练好，可以通过增大epochs和lr来解决这个问题。

---

# A1 实验心得

学会手动构建前馈神经网络和利用torch.nn构建前馈神经网络解决回归、二分类、和多分类问题
1. 实验中发现学习率的设置至关重要，如果学习率过大则会导致准确率下降的趋势，若学习率过小会导致模型需要更多时间收敛
2. 实验过程中发现出现过拟合现象，通过修改相关参数得以纠正
3. 学会程序模块话的编写，避免重复编写代码
4. 对激活函数的选取有了更加清晰的认识
5. 隐藏层的个数和隐藏层的神经元个数对模型有着很大的影响。

# A2 参考文献  
参考课程PPT